In [1]:
import assets.helper as b3
import assets.functions as run


In [2]:
import pandas as pd
import numpy as np
import os

import assets.graphs as g


### QUOTES PRICES

In [ ]:
fund = run.load_pkl(f'{b3.app_folder}/fund')
quotes = run.load_pkl(f'{b3.app_folder}/quotes')


In [ ]:
fund.keys(), fund['BENS INDUSTRIAIS'].columns

In [ ]:
quotes.keys(), quotes['AERIS'].keys()

In [ ]:
df_preplot = {}
start_time = run.time.time()

for i, (setor, df_fund) in enumerate(fund.items()):
    df_fund = preprocess_data(df_fund)
    df_cd_conta = pivot_data(df_fund)

    df_unique = df_fund.drop_duplicates(subset=['DT_REFER', 'PREGAO']).drop(['CD_CONTA', 'DS_CONTA', 'VL_CONTA'], axis=1)
    df_premerged = pd.merge(df_unique, df_cd_conta, on=['DT_REFER', 'PREGAO'])
    df_resampled = resample_data(df_premerged)
    
    df_merged = merge_with_bigdata(df_resampled, bigdata)
    df_merged = df_merged.set_index('Date', drop=True)
    df_merged = df_merged.groupby('PREGAO', group_keys=False).apply(add_metrics)

    # Clean up the dataframe using the cleanup_dataframe function
    df_merged = cleanup_dataframe(df_merged)

    df_merged = add_metrics(df_merged)

    df_preplot[setor] = df_merged

    print(setor, run.remaining_time(start_time, len(fund), i))


In [ ]:
# Define the path to the folder
company_folder = 'company'
folder_path = os.path.join(b3.app_folder, company_folder)

# Check if the folder does not exist
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)


In [ ]:
start_time = run.time.time()
for i, (setor, df) in enumerate(df_preplot.items()):
    companies = df['PREGAO'].unique()
    for i2, company in enumerate(companies):
        mask = df['PREGAO'] == company
        df_temp = df[mask]
        try:
            df_temp = run.save_pkl(df_temp, f'{b3.app_folder}/{company_folder}/{company}')
        except Exception as e:
            pass
    print(setor, run.remaining_time(start_time, len(df_preplot), i))


In [ ]:
mask = df_preplot['BENS INDUSTRIAIS']['TICKER'] == 'AERI3'
df_preplot['BENS INDUSTRIAIS'][mask]

### DASH

In [3]:
company = 'AZEVEDO'
df = run.load_pkl(f'{b3.app_folder}/{b3.company_folder}/{company}')
tickers = df['TICKER'].unique()
# df.columns.to_list()

In [ ]:
df['03.11 - Lucro Líquido']
df['14.04.01 - ROE (Resultado por Patrimônio)']
df['03.01 - Receita Bruta']
df['06.01 - Caixa das Operações']





In [ ]:
graphs = g.construct_graphs(df)
graphs.keys()

In [ ]:
import assets.lines
lines = assets.lines.report
groups = set(entry['title'][:2] for entry in lines)
groups

In [ ]:
for g, (group_key, group) in enumerate(graphs.items()):
    status = True if g == 0 else False
    plots = []
    for l, (line_key, line) in enumerate(group.items()):
        for p, (plot_key, plot_info) in enumerate(line.items()):
            for ticker in df['TICKER'].unique():
                df_ticker = df[df['TICKER'] == ticker]
                if not df.empty:
                    print(group[0][0]['info']['title'])
                    print(line[0]['info']['title'])
                    print(plot_info['info']['title'])
                    run.plot_tweak(plot_info, df_ticker).show();

In [ ]:
for ticker in tickers:
    df = df[df['TICKER'] == ticker]
    print(ticker, len(df))

    if not df.empty:
        for g, (group_key, group) in enumerate(merged_entries.items()):
            status = True if g == 0 else False
            plots = []
            for l, (line_key, line) in enumerate(group.items()):
                for p, (plot_key, plot_info) in enumerate(line.items()):
                    print(ticker)
                    run.plot_tweak(plot_info, df).show();

In [ ]:
fund = run.load_database()


In [ ]:
setores = []
for setor, df in fund.items():
    columns=['SETOR', 'SUBSETOR', 'SEGMENTO', 'PREGAO']
    setores.append(df[columns].drop_duplicates())
    print(setor)
sss = pd.concat(setores)
sss = sss.applymap(run.clean_text)

sss = run.save_pkl(sss, f'{b3.app_folder}/sss')

In [ ]:
sss

In [ ]:
mask = sss['SUBSETOR'] == 'MATERIAL DE TRANSPORTE'
sss[mask]

### Playground


In [8]:
company_list = run.load_pkl('company_list')
company_list

,COMPANHIA,PREGAO,TICK,LISTAGEM
0,1461 INVESTIMENTOS SA,1461 INVEST,UQMU,
1,3R PETROLEUM OLEO E GAS SA,3R PETROLEUM,RRRP,NM
2,3R POTIGUAR SA,3RPOTIGUAR,PTGU,
3,3Z REALTY DESENVOLVIMENTO IMOBILIARIO SA,3ZREALTY,TZRD,
4,524 PARTICIPACOES SA,524 PARTICIP,QVQP,MB
...,...,...,...,...
2558,YUNY INCORPORADORA SA,YUNY,YUNY,
2559,ZA DIGITAL SP SIST ESTACIONAMENTO ROTARIVO SA,Z.A DIGITAL,ZZDT,
2560,ZAMP SA,ZAMP S.A.,ZAMP,NM
2561,ZEMA CIA DE PETROLEO,ZEMA,ZMPT,


In [6]:
listagem_siglas = ['NM', 'N1', 'N2', 'MA', 'M2', 'MB', 'DR1', 'DR2', 'DR3', 'DRE', 'DRN']
listagem_extenso = ['Novo Mercado', 'Nível 1 de Governança Corporativa', 'Nível 2 de Governança Corporativa', 'Bovespa Mais', 'Bovespa Mais Nível 2', 'Organização Tradicional OTC', 'BDR Nível 1', 'BDR Nível 2', 'BDR Nível 3', 'BDR ETF', 'BDR Não Patrocinado']
list_dict = dict(zip(listagem_siglas, listagem_extenso))
list_dict

{'NM': 'Novo Mercado',
 'N1': 'Nível 1 de Governança Corporativa',
 'N2': 'Nível 2 de Governança Corporativa',
 'MA': 'Bovespa Mais',
 'M2': 'Bovespa Mais Nível 2',
 'MB': 'Organização Tradicional OTC',
 'DR1': 'BDR Nível 1',
 'DR2': 'BDR Nível 2',
 'DR3': 'BDR Nível 3',
 'DRE': 'BDR ETF',
 'DRN': 'BDR Não Patrocinado'}

In [43]:
companies_from_file = run.load_pkl('companies_from_file')
b3_companies_tickers = run.load_pkl('b3_companies_tickers')
new_companies = run.load_pkl('new_companies')
companies_from_file.columns.to_list()

['pregao',
 'company_name',
 'cvm',
 'listagem',
 'ticker',
 'tickers',
 'asin',
 'cnpj',
 'site',
 'setor',
 'subsetor',
 'segmento',
 'atividade',
 'escriturador',
 'url']

In [38]:
def find_columns_with_dfs(df):
    columns_with_dfs = []
    for column in df.columns:
        # Check if any cell in this column is a DataFrame
        if any(isinstance(cell, pd.DataFrame) for cell in df[column]):
            columns_with_dfs.append(column)
    return columns_with_dfs

# Usage
columns_with_dfs = find_columns_with_dfs(new_companies)
columns_with_dfs
new_companies[columns_with_dfs] = ''

In [41]:
# Merge the two DataFrames
merged_companies = pd.concat([companies_from_file.iloc[:, :-1], new_companies.iloc[:, :-1]]).drop_duplicates().fillna('')

# # Convert cvm_code to integer for proper sorting
# merged_companies['cvm_code'] = merged_companies['cvm_code'].astype(int)

# # Sort by cvm_code
# sorted_companies = merged_companies.sort_values(by='cvm_code')

# sorted_companies.reset_index(drop=True, inplace=True)


In [42]:
merged_companies

,pregao,company_name,cvm,listagem,ticker,tickers,asin,cnpj,site,setor,subsetor,segmento,atividade,escriturador,url,cvm_code,companhia,stock_holders,name,isins
0,2W ENERGIA,2W ENERGIA SA,917581,,2WAV,,,,,,,,,,,,,,,
1,3R PETROLEUM,3R PETROLEUM OLEO E GAS SA,25291,,RRRP,RRRP3,BRRRRPACNOR5,12.091.809/0001-55,,Petróleo. Gás e Biocombustíveis,Petróleo. Gás e Biocombustíveis,Exploração. Refino e Distribuição,Exploração e produção de petróleo e gás natural.,ITAU CORRETORA ACOES,,,,,,
2,3ZREALTY,3Z REALTY DESENVOLVIMENTO IMOBILIARIO SA,908830,,TZRD,,,08.145.855/0001-12,,,,,,,,,,,,
3,524 PARTICIP,524 PARTICIPACOES SA,16284,,QVQP,QVQP3B,BRQVQPACNOR1,01.851.771/0001-55,http://www.524participacoes.com.br/home/index....,Outros,Outros,Outros,Holding - participação em outras sociedades,ITAU CORRETORA ACOES,,,,,,
4,GERADORA,A GERADORA ALUGUEL DE MAQUINAS SA,904110,,GRDR,,,33.845.322/0001-90,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,VTLA,VOLTALIA SAO MIGUEL DO GOSTOSO PARTICIPACOES SA,,,,,,19.943.730/0001-54,,Nenhum,Nenhum,Nenhum,,,https://sistemaswebb3-listados.b3.com.br/liste...,909465,,,SAOMIGUEL,
0,VTCT,VOTORANTIM CIMENTOS NNE SA,,,,,,10.656.452/0001-80,,Nenhum,Nenhum,Nenhum,,,https://sistemaswebb3-listados.b3.com.br/liste...,909366,,,VOTOCIMENTOS,
0,WDPR,WINDEPAR HOLDING S A,,,,,,23.733.733/0001-30,,Nenhum,Nenhum,Nenhum,,,https://sistemaswebb3-listados.b3.com.br/liste...,909507,,,WINDEPAR,
0,XNIC,XNICE PARTICIPACOES SA,,,,,,17.426.229/0001-95,,Nenhum,Nenhum,Nenhum,,,https://sistemaswebb3-listados.b3.com.br/liste...,909606,,,XNICEPART,
